I will attemp to make a visualization of the COVID pandemic. I will use the Bokeh visualizaion library.

Data source [Our World in Data](https://covid.ourworldindata.org/data/owid-covid-data.csv)

[Additional data](https://covid.ourworldindata.org/data/ecdc/locations.csv) on locations.

[Bokeh](https://github.com/bokeh/bokeh) is an open source project published under the [BSD](https://en.wikipedia.org/wiki/BSD_licenses#:~:text=BSD%20licenses%20are%20a%20family,and%20distribution%20of%20covered%20software.&text=The%20original%20BSD%20license%20was,a%20Unix%2Dlike%20operating%20system.) license.

You can find a nice presentation [here](https://www.youtube.com/watch?v=O5OvOLK-xqQ) from one of the developers Sarah Bird.

From their GitHub:

Bokeh is an interactive visualization library for modern web browsers. It provides elegant, concise construction of versatile graphics, and affords high-performance interactivity over large or streaming datasets. Bokeh can help anyone who would like to quickly and easily make interactive plots, dashboards, and data applications.

As a side note. This came up in a quick google searc for declarative vs. imperative. Declarative programming is when you say what you want, and imperative language is when you say how to get what you want.

The way Bokeh works is, that it gets bundled into a json file wich gets pushed to a client, were it gets rendered as a javasript. (There is a BokehJS library)

Visualization Technique (25%)
A narrative description of the visualization you are planning to use, describing how it works
A discussion of in which circumstances this visualization should and should not be used (what is is close to? What else could you consider? How does it relate to specific aspects of data?

# Dependencies

* PyYAML>=3.10
* python-dateutil>=2.1
* Jinja2>=2.7
* numpy>=1.11.3
* pillow>=4.0
* packaging>=16.8
* tornado>=5
* typing_extensions >=3.7.4

~~~
conda install -c anaconda pyyaml
conda install -c anaconda python-dateutils
conda install -c conda-forge jinja2
conda install -c conda-forge numpy
conda install -c conda-forge pillow
conda install -c conda-forge packaging
conda install -c conda-forge tornado
conda install -c conda-forge typing_extensions
~~~

for pip users just use pip install

* Optional
~~~
conda install -c conda-forge scripting
~~~
# Instalation
With pip
~~~
pip install bokeh
~~~
With conda
~~~
conda install bokeh
~~~

Make sure to close all notebook instances before installing dependencies.
~~~~
jupyter labextension install @jupyter-widgets/jupyterlab-manager

jupyter labextension install @bokeh/jupyter_bokeh
~~~~

Bokeh Is used to make interactive visualizations for the web wich is an area that interest me. It offers server-side processing for big data wich is also something I would like to explore for my own work.



Visualization Library (25%)
The library you are going to use, and a background on why the library is good for this visualization. Who created it? Is it open source? How do you install it?
A discussion of the general approach and limitations of this library. Is it declarative or procedural? Does it integrate with Jupyter? Why you decided to use this library (especially if there are other options)?

Demonstration (50%)
The dataset you picked and instructions for cleaning the dataset. You should pick a suitable dataset to demonstrate the technique, toolkit, and problem you are facing. 
The quality of your demonstration. First demonstrate the basics of this approach, then show a few of the edges of how the library might be used for other cases. This is the "meat" of the assignment.

In [9]:
%matplotlib inline

from urllib.request import urlretrieve
from datetime import date, datetime, timedelta
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import json
from bokeh.io import show, curdoc, output_notebook
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.models import LinearColorMapper, LogColorMapper, Slider
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.palettes import Greys256, Inferno256, Magma256, Plasma256

In [10]:
covid_data_url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
location_data_url = 'https://covid.ourworldindata.org/data/ecdc/locations.csv'

def get_data(url, filename, force_download=False):
    """Download and cache data

    Parameters
    ----------
    filename : string
    url : string -> web location of data
    force_download : bool -> if True, force download of data

    Returns
    -------
    response - file with filename to current directory
    """

    if force_download or not os.path.exists(filename):
       	res  = urlretrieve(url, filename)
    return None

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code
    jgbarah on Stackoverflow"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

In [11]:
get_data(covid_data_url, 'CovidData.csv', force_download=False)

covid = pd.read_csv('CovidData.csv', index_col='date', parse_dates=True)
geo = gpd.read_file('ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')

covid = covid.reset_index()
covid = covid.fillna(1)
covid = covid[covid.date.isnull() == False]

geo = geo[['ADM0_A3','geometry']]
geo.rename(columns={'ADM0_A3':'iso_code'}, inplace=True)


merged = geo.merge(covid, on='iso_code', how='outer')
merged_start = merged[merged.date == merged.iloc[0].date].dropna()

In [12]:
def get_last_day(merged):
    return merged.date.max()

def get_first_day(merged):
    return merged.date.min()
# Input GeoJSON source that contains features for plotting

low = merged.total_cases_per_million.min()
high = merged.total_cases_per_million.max()
# Lets choose a suitable palette (inferno fits quite god for 2020)


def json_data(day_of_year):    
    td = timedelta(days=day_of_year)
    df_day = merged[merged.date == merged.iloc[0].date + td].dropna()
    geosource = df_day.to_json(default=json_serial)
    return geosource

def update_plot(attr, old, new):
    day_of_year = int(slider.value)
    new_data = json_data(day_of_year)
    geosource.geojson = new_data
    p.title.text = 'Covid cases per million on'

palette = Inferno256[::-1]
color_mapper = LinearColorMapper(palette=palette, low=float(low), high=float(high))

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff = 8,
                     width = 20, height = 500,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'vertical')
geosource = GeoJSONDataSource(geojson = json_data(1)) 
# Create figure object.
p = figure(title = 'Covid cases per million', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field':'total_cases_per_million', 'transform':color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
#Create hover tool
p.add_tools(HoverTool(tooltips = [('Country','@location'),('Total Cases','@total_cases')]))
p.add_layout(color_bar, 'right')

slider = Slider(title='day of 2020', start=1, end=365, step=1, value=1)
slider.on_change('value', update_plot)

layout = column(p, widgetbox(slider))
curdoc().add_root(layout)

output_notebook()
show(layout)

# GeoJSON is a popular open standard for representing geographical features with JSON. It describes points, lines and polygons (called Patches in Bokeh)
# as a collection of features. Each feature can also have a set of properties.
# We therefor convert our data to JSON object

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

